<a href="https://colab.research.google.com/github/hanumanthsistla/NER-Flask-TensorFlow/blob/master/RL_Cartpole_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Classic cart-pole system implemented by Rich Sutton et al.
Copied from http://incompleteideas.net/sutton/book/code/pole.c
permalink: https://perma.cc/C9ZM-652R
"""
import math
from typing import Optional, Union

import numpy as np

import gymnasium
from gymnasium import logger, spaces
from gymnasium.envs.classic_control import utils
from gymnasium.error import DependencyNotInstalled

In [ ]:
!pip install stable-baselines3[extra]

In [ ]:
import os
import gymnasium
from stable_baselines3 import PPO

# stable baseline3 is allow vectorize environment in which agent learn on more than one environment at the same time
from stable_baselines3.common.vec_env import DummyVecEnv
# for test the model performance
from stable_baselines3.common.evaluation import evaluate_policy

import warnings
warnings.filterwarnings("ignore")

In [ ]:
environment_name = "CartPole-v1"

In [ ]:
env = gymnasium.make(environment_name)

In [ ]:
episodes = 5

for episode in range(1, episodes + 1):
    state = env.reset()  # initial set of observation
    done = False
    score = 0

    while not done:
        env.render()    # display the environment
        action = env.action_space.sample()
        step_result = env.step(action)

        # Unpack the first four elements, ignore the rest
        n_state, reward, done, info, _ = step_result
        score += reward

    print('Episode:{} Score:{}'.format(episode, score))

env.close()

Episode:1 Score:12.0
Episode:2 Score:17.0
Episode:3 Score:29.0
Episode:4 Score:10.0
Episode:5 Score:30.0


In [ ]:
env.reset()

(array([-0.04754717, -0.00600237,  0.04836069,  0.00997843], dtype=float32),
 {})

In [ ]:
# Action space is '0' or '1'

env.action_space

Discrete(2)

In [ ]:
# 0-push cart to left, 1-push cart to the right
env.action_space.sample()

np.int64(0)

In [ ]:
# 0-push cart to left, 1-push cart to the right
env.action_space.sample()

np.int64(1)

In [ ]:
# 0-push cart to left, 1-push cart to the right
env.action_space.sample()

np.int64(1)

In [ ]:
env.observation_space

Box([-4.8               -inf -0.41887903        -inf], [4.8               inf 0.41887903        inf], (4,), float32)

In [ ]:
# [cart position[-4.8,4.8], cart velocity[-Inf,Inf], pole angle[-24,24], pole angular velocity[-Inf,Inf]]
env.observation_space.sample()

array([ 0.18024808,  0.05142455, -0.22693717,  1.2710905 ], dtype=float32)

In [ ]:
env.step(1)

(array([-0.04766721,  0.18839386,  0.04856026, -0.26706252], dtype=float32),
 1.0,
 False,
 False,
 {})

In [ ]:
log_path=os.path.join('Training','Logs')
log_path

'Training/Logs'

In [ ]:
env = gymnasium.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1,tensorboard_log=log_path)

Using cpu device


Stable Baselines has three policy:

    MlpPolicy
    CnnPolicy
    MultiInputPolicy


In [ ]:
model.learn(total_timesteps=20000)

Logging to Training/Logs/PPO_1
-----------------------------
| time/              |      |
|    fps             | 1082 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 786         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008273961 |
|    clip_fraction        | 0.0824      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.687      |
|    explained_variance   | -0.0015     |
|    learning_rate        | 0.0003      |
|    loss                 | 6.57        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0134     |
|    value_loss           | 45.9        |
-----------------------------------------
---

In [ ]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model')

In [ ]:
model.save(PPO_path)

In [ ]:
PPO_path

'Training/Saved Models/PPO_model'

In [ ]:
del model

In [ ]:
model = PPO.load(PPO_path, env=env)

In [ ]:
model.learn(total_timesteps=1000)

Logging to Training/Logs/PPO_2
-----------------------------
| time/              |      |
|    fps             | 1286 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------


In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy
evaluate_policy(model, env, n_eval_episodes=10, render=True)
# return average reward and standard deviation

(np.float64(500.0), np.float64(0.0))

In [ ]:
env.close()

In [ ]:

# Test the Cartpole Model

episodes = 5
for episode in range(1, episodes + 1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action, _ = model.predict(obs) # now using model here
        step_result = env.step(action)

        obs, reward, done, info = step_result
        score += reward

    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:[500.]
Episode:2 Score:[150.]
Episode:3 Score:[294.]
Episode:4 Score:[500.]
Episode:5 Score:[500.]


In [ ]:
env.close()

In [ ]:
obs=env.reset()

In [ ]:
obs

array([[-0.04597888,  0.03789401,  0.00045879, -0.01810813]],
      dtype=float32)

In [ ]:
action, _ = model.predict(obs)

In [ ]:
env.action_space.sample()

np.int64(0)

In [ ]:
env.step(action)

(array([[-4.5220997e-02, -1.5723452e-01,  9.6626274e-05,  2.7471951e-01]],
       dtype=float32),
 array([1.], dtype=float32),
 array([False]),
 [{'TimeLimit.truncated': False}])

In [ ]:
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done:
        print('info', info)
        break

info [{'TimeLimit.truncated': True, 'terminal_observation': array([-1.3993659 , -0.41161793,  0.10521293,  0.1951611 ], dtype=float32)}]


In [ ]:
env.close()

In [ ]:
training_log_path = os.path.join(log_path, 'PPO_2')

In [ ]:
training_log_path

'Training/Logs/PPO_2'

In [ ]:
# Adding a callback to the training Stage
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os

In [ ]:
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')

In [ ]:
env = gymnasium.make(environment_name)
env = DummyVecEnv([lambda: env])

In [ ]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1)
eval_callback = EvalCallback(env,
                             callback_on_new_best=stop_callback,
                             eval_freq=10000,
                             best_model_save_path=save_path,
                             verbose=1)

In [ ]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training/Logs/PPO_3
-----------------------------
| time/              |      |
|    fps             | 1295 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 963         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007917931 |
|    clip_fraction        | 0.0848      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00828    |
|    learning_rate        | 0.0003      |
|    loss                 | 8.67        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0124     |
|    value_loss           | 58.4        |
-----------------------------------------
---

In [ ]:
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(np.float64(317.0), np.float64(128.34328965707556))

In [ ]:
env.close()

In [ ]:
# Changing Policies
# new neural netwok architecture define
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [ ]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path, policy_kwargs={'net_arch': net_arch})

Using cpu device


In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training/Logs/PPO_4
-----------------------------
| time/              |      |
|    fps             | 1003 |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 707        |
|    iterations           | 2          |
|    time_elapsed         | 5          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.01515099 |
|    clip_fraction        | 0.21       |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.681     |
|    explained_variance   | -0.00288   |
|    learning_rate        | 0.0003     |
|    loss                 | 1.99       |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.0217    |
|    value_loss           | 17.4       |
----------------------------------------
---------------------

In [ ]:
# Using alternate algorithm

from stable_baselines3 import DQN

In [ ]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training/Logs/DQN_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.947    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 2217     |
|    time_elapsed     | 0        |
|    total_timesteps  | 112      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.545    |
|    n_updates        | 2        |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.907    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 1632     |
|    time_elapsed     | 0        |
|    total_timesteps  | 195      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.567    |
|    n_updates        | 23       |
----------------------------------
----------------------------------
| rollout/            | 

In [ ]:
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')

In [ ]:
model.save(dqn_path)

In [ ]:
model = DQN.load(dqn_path, env=env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(np.float64(35.6), np.float64(4.608687448721165))

In [ ]:
env.close()